# Test XTTS Model

## Improt Moduels 

In [1]:
import os
import torch
import torchaudio
from TTS.tts.configs.xtts_config import XttsConfig
from TTS.tts.models.xtts import Xtts
    

## CONST VARIBALES

In [51]:
TEXT_SENTENCE = "أهلا أنا صاي مساعدك الرقمي أقدر أساعدك أزاي"
LANGUAGE = "ar"
TEMPERATURE = 0.60


## LOAD CHEKPT

In [45]:
CONFIG_PATH = "../models/new_test/ckpt_test/config.json"
TOKENIZER_PATH = "../models/new_test/ckpt_test/vocab.json"
XTTS_CHECKPOINT = "../models/new_test/ckpt_test/best_model_24795.pth"
CHKPT_DIR = "../models/new_test/ckpt_test"

## LOAD SPEAKER REFERENCE

In [35]:
SPEAKER_REFERENCE = "/home/abdelrahman-khaled/Projects/SpeechProjects/SeqoonTTS/Data/EGY_conv_speech_corpus/wavs/A0003_S002_0_G0002_077.wav"

## OUTPUT PATH 

In [52]:
OUTPUT_WAV_PATH = "/home/abdelrahman-khaled/Projects/SpeechProjects/SeqoonTTS/wavs/model_outputs/example_output_2.wav"

## INFRANCE  

In [53]:
print("Loading model...")
config = XttsConfig()
config.load_json(CONFIG_PATH)
model = Xtts.init_from_config(config)
model.load_checkpoint(config, checkpoint_path=XTTS_CHECKPOINT, vocab_path=TOKENIZER_PATH, use_deepspeed=False, speaker_file_path="", checkpoint_dir=CHKPT_DIR)
model.cuda()

print("Computing speaker latents...")
gpt_cond_latent, speaker_embedding = model.get_conditioning_latents(audio_path=[SPEAKER_REFERENCE])

print("Inference...")
out = model.inference(
        text=TEXT_SENTENCE,
        language="ar",  # Arabic
        gpt_cond_latent=gpt_cond_latent,
        speaker_embedding=speaker_embedding,
        temperature=0.65,                     #range-> 0.55:0.75
        length_penalty=1.0,
        repetition_penalty=2.0,
        top_k=50,
        top_p=0.8,
        speed=1.0,
)


Loading model...
Computing speaker latents...
Inference...


## SAVE WAV

In [54]:
torchaudio.save(OUTPUT_WAV_PATH, torch.tensor(out["wav"]).unsqueeze(0), 24000)

## DISPLAY WAV

In [ ]:
import IPython
IPython.display.Audio("")

## DOWNLOAD ORIGINAL CKPT  

In [ ]:
!wget https://github.com/coqui-ai/TTS/releases/download/speaker_encoder_model/config_se.json
!wget https://github.com/coqui-ai/TTS/releases/download/speaker_encoder_model/model_se.pth.tar

## INFRANCE ORIGINAL MODEL

In [ ]:
!tts --model_name tts_models/multilingual/multi-dataset/xtts_v2 \
--encoder_path model_se.pth.tar \
--encoder_config config_se.json \   
--speaker_wav ../SPEAKER_REF_WAVS/LJ001-0001.wav \
--text "123 ممكن تِزودْني باسمْ المركزْ اللي حابْ أحْجزْ لكْ فيهْ"\
--out_path ../wavs/model_outputs/spkr-old20.wav \
--language_idx "ar"

## DISPLAY WAV

In [ ]:
import IPython
IPython.display.Audio("")